In [14]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

from textblob import TextBlob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tweepy
import json
import datetime
import time
from datetime import datetime, timedelta
#import re

In [10]:
US_list=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
      'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN',
      'TX','UT','VT','VA','WA','WV','WI','WY']

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# set token and key
consumer_key = "8vFDLTHTFkspg4tBUGnJfL2v7"
consumer_secret = "qUxjq3eafmP5MrNbfrFVLcSIJUVgg8O14Hm05fV2TJIX2Kl866"
access_token = "889613119043108865-4q72TLUEYN3wWS1UPLjU0SmzAhEIHy3"
access_token_secret = "F9oX2tZH07Mvp66wM0N48q12eb9RD7uYbV8DIOUms6F87"

In [5]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


In [9]:
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        df['location'] = np.array([tweet.user.location for tweet in tweets])
        
        return df

In [6]:
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        return auth

In [7]:
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    
        
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list,user_id,location,create_time,text,time_limit):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename,user_id,location,create_time,text,time_limit)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [8]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename,tweets,location,create_time,text,time_limit):
        self.fetched_tweets_filename = fetched_tweets_filename
        self.user_id=user_id
        self.location=location
        self.create_time=create_time
        self.text=text
        
        self.start_time = time.time()
        self.limit = time_limit
        self.saveFile = open('20200220_9PM.json', 'a')
        super(TwitterListener, self).__init__()
        
    def on_data(self, data):
        try:
            if (time.time() - self.start_time) < self.limit:
                js=json.loads(data)
                id=js["user"]["id"]
                self.user_id.append(id)
                user_location=js["user"]["location"]
                self.location.append(user_location)
                tm=js["created_at"]
                ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tm,'%a %b %d %H:%M:%S +0000 %Y'))
                tt=datetime.strptime(ts,'%Y-%m-%d %H:%M:%S')
                tweet_time = tt- timedelta(hours=5)
                tweet_time.strftime('%Y-%m-%d %H:%M:%S')
                self.create_time.append(tweet_time)
                tweet_text=js['text']
                self.text.append(tweet_text)
                #self.tweets.append({'User_ID': user_id, 'Location': user_location, 'Created_time': tweet_time, 'Text':tweet_text}, ignore_index=True)
                self.saveFile.write(data)
                self.saveFile.write('\n')
                return True
            else:
                self.saveFile.close()
                return False
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


In [50]:
if __name__ == '__main__':

    #twitter_client = TwitterClient()
    #tweet_analyzer = TweetAnalyzer()

    #api = twitter_client.get_twitter_client_api()

    #tweets = api.user_timeline(screen_name="realDonaldTrump", count=20)
    #print(type(tweets))
    #tweets= TwitterStreamer.stream_tweets()
    
    #twitter_client = TwitterClient('pycon')
    #print(twitter_client.get_user_timeline_tweets(1))
    
    # Authenticate using config.py and connect to Twitter Streaming API.
    hash_tag_list = ["flu", "got flu", "influenza","catch flu"]
    fetched_tweets_filename = "tweets.json"
    tweets = pd.DataFrame(columns=['User_ID','Location', 'Create_time', 'Text'])
    
    user_id=[]
    location=[]
    create_time=[]
    text=[]
    time_limit=25000
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list,user_id,location,create_time,text,time_limit)
    
    df = pd.DataFrame({'User_ID': user_id, 'Location': location, 'Created_time': create_time, 'Text':text})
    df.loc[df['Location'].str.contains('AL|Alabama|Yellowhammer ', na=False), 'Location'] = 'AL'
    df.loc[df['Location'].str.contains('AK|Alaska|Last Frontier', na=False), 'Location'] = 'AK'
    df.loc[df['Location'].str.contains('AZ|Arizona|Grand Canyon', na=False), 'Location'] = 'AZ'
    df.loc[df['Location'].str.contains('AR|Arkansas|Natural', na=False), 'Location'] = 'AR'
    df.loc[df['Location'].str.contains('CA|California|Golden', na=False), 'Location'] = 'CA'
    df.loc[df['Location'].str.contains('CO|Colorado|Centennial', na=False), 'Location'] = 'CO'
    df.loc[df['Location'].str.contains('CT|Connecticut|Nutmeg|Constitution', na=False), 'Location'] = 'CT'
    df.loc[df['Location'].str.contains('DE|Delaware|First|Diamond', na=False), 'Location'] = 'DE'
    df.loc[df['Location'].str.contains('FL|Florida|Sunshine', na=False), 'Location'] = 'FL'
    df.loc[df['Location'].str.contains('GA|Georgia|Peach', na=False), 'Location'] = 'GA'
    df.loc[df['Location'].str.contains('HI|Hawaii|Aloha', na=False), 'Location'] = 'HI'
    df.loc[df['Location'].str.contains('ID|Idaho|Gem', na=False), 'Location'] = 'ID'
    df.loc[df['Location'].str.contains('IL|Illinois|Prairie', na=False), 'Location'] = 'IL'
    df.loc[df['Location'].str.contains('IN|Indiana|Hoosier', na=False), 'Location'] = 'IN'
    df.loc[df['Location'].str.contains('IA|Iowa|Hawkeye|Corn', na=False), 'Location'] = 'IA'
    df.loc[df['Location'].str.contains('KS|Kansas|Sunflower', na=False), 'Location'] = 'KS'
    df.loc[df['Location'].str.contains('KY|Kentucky|Bluegrass', na=False), 'Location'] = 'KY'
    df.loc[df['Location'].str.contains('LA|Louisiana|Pelican', na=False), 'Location'] = 'LA'
    df.loc[df['Location'].str.contains('ME|Maine|Vacationland', na=False), 'Location'] = 'ME'
    df.loc[df['Location'].str.contains('MD|Maryland|Old Line', na=False), 'Location'] = 'MD'
    df.loc[df['Location'].str.contains('MA|Massachusetts|Bay', na=False), 'Location'] = 'MA'
    df.loc[df['Location'].str.contains('MI|Michigan|Great Lake|Wolverine', na=False), 'Location'] = 'MI'
    df.loc[df['Location'].str.contains('MN|Minnesota|North Star', na=False), 'Location'] = 'MN'
    df.loc[df['Location'].str.contains('MS|Mississippi|Magnolia', na=False), 'Location'] = 'MS'
    df.loc[df['Location'].str.contains('MO|Missouri|Show Me', na=False), 'Location'] = 'MO'
    df.loc[df['Location'].str.contains('MT|Montana|Treasure', na=False), 'Location'] = 'MT'
    df.loc[df['Location'].str.contains('NE|Nebraska|Cornhusker', na=False), 'Location'] = 'NE'
    df.loc[df['Location'].str.contains('NV|Nevada|Silver', na=False), 'Location'] = 'NV'
    df.loc[df['Location'].str.contains('NH|New Hampshire|Granite', na=False), 'Location'] = 'NH'
    df.loc[df['Location'].str.contains('NJ|New Jersey|Garden', na=False), 'Location'] = 'NJ'
    df.loc[df['Location'].str.contains('NM|New Mexico|Sunshine|Enchantment', na=False), 'Location'] = 'NM'
    df.loc[df['Location'].str.contains('NY|New York|Empire', na=False), 'Location'] = 'NY'
    df.loc[df['Location'].str.contains('NC|North Carolina|Tar Heel', na=False), 'Location'] = 'NC'
    df.loc[df['Location'].str.contains('ND|North Dakota|Flickertail', na=False), 'Location'] = 'ND'
    df.loc[df['Location'].str.contains('OH|Ohio|Buckeye', na=False), 'Location'] = 'OH'
    df.loc[df['Location'].str.contains('OK|Oklahoma|Sooner', na=False), 'Location'] = 'OK'
    df.loc[df['Location'].str.contains('OR|Oregon|Beaver', na=False), 'Location'] = 'OR'
    df.loc[df['Location'].str.contains('PA|Pennsylvania|Keystone', na=False), 'Location'] = 'PA'
    df.loc[df['Location'].str.contains('RI|Rhode Island|Ocean', na=False), 'Location'] = 'RI'
    df.loc[df['Location'].str.contains('SC|South Carolina|Palmetto', na=False), 'Location'] = 'SC'
    df.loc[df['Location'].str.contains('SD|South Dakota|Mount Rushmore', na=False), 'Location'] = 'SD'
    df.loc[df['Location'].str.contains('TN|Tennessee|Volunteer', na=False), 'Location'] = 'TN'
    df.loc[df['Location'].str.contains('TX|Texas|Lone Star', na=False), 'Location'] = 'TX'
    df.loc[df['Location'].str.contains('UT|Utah|Beehive', na=False), 'Location'] = 'UT'
    df.loc[df['Location'].str.contains('VT|Vermont|Green Mountain', na=False), 'Location'] = 'VT'
    df.loc[df['Location'].str.contains('VA|Virginia|Old Dominion', na=False), 'Location'] = 'VA'
    df.loc[df['Location'].str.contains('WA|Washington|DC|Evergreen', na=False), 'Location'] = 'WA'
    df.loc[df['Location'].str.contains('WV|West Virginia|Mountain ', na=False), 'Location'] = 'WV'
    df.loc[df['Location'].str.contains('WI|Wisconsin|Badger', na=False), 'Location'] = 'WI'
    df.loc[df['Location'].str.contains('WY|Wyoming|Equality', na=False), 'Location'] = 'WY'

    #print(df)
    
    Valid_df = df[df['Location'].isin(US_list)]

    print(Valid_df)
    
    export_csv = Valid_df.to_csv ('data_0220.csv',mode='a', index = None, header=False,encoding='utf_8_sig') 
    
    #df_ = tweet_analyzer.tweets_to_data_frame(tweets)
    #sentiment

    #df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    #print(df.head(10))
    
    
    #print(dir(tweets[0]))
    #print(tweets[0].retweet_count)


    # Get average length over all tweets:
    #print(np.mean(df['len']))

    # Get the number of likes for the most liked tweet:
    #print(np.max(df['likes']))

    # Get the number of retweets for the most retweeted tweet:
    #print(np.max(df['retweets']))
    
   # print(df.head(10))
    
    # Time Series
    #time_likes = pd.Series(data=df['len'].values, index=df['date'])
    #time_likes.plot(figsize=(16, 4), color='r')
    #plt.show()
    
    #time_favs = pd.Series(data=df['likes'].values, index=df['date'])
    #time_favs.plot(figsize=(16, 4), color='r')
    #plt.show()

    #time_retweets = pd.Series(data=df['retweets'].values, index=df['date'])
    #time_retweets.plot(figsize=(16, 4), color='r')
    #plt.show()

    # Layered Time Series:
    #time_likes = pd.Series(data=df['likes'].values, index=df['date'])
    #time_likes.plot(figsize=(16, 4), label="likes", legend=True)

    #time_retweets = pd.Series(data=df['retweets'].values, index=df['date'])
    #time_retweets.plot(figsize=(16, 4), label="retweets", legend=True)
    #plt.show()

                   User_ID Location        Created_time  \
2               1172663630       CA 2020-02-21 02:44:29   
9                232450395       CA 2020-02-21 02:45:06   
20               715479965       NC 2020-02-21 02:45:43   
21               715479965       NC 2020-02-21 02:45:44   
24     1143019564646588416       AR 2020-02-21 02:45:48   
28      765987939163762688       FL 2020-02-21 02:45:59   
43     1083425001905287169       AL 2020-02-21 02:46:17   
44      715957219972530180       CA 2020-02-21 02:46:21   
51      941100580386295808       CA 2020-02-21 02:46:52   
52               529385469       NY 2020-02-21 02:46:54   
58                37076194       IA 2020-02-21 02:47:11   
64      783397310508204032       MN 2020-02-21 02:47:32   
65               144519030       MT 2020-02-21 02:47:34   
75               251448803       AZ 2020-02-21 02:48:04   
79               429494446       TX 2020-02-21 02:48:21   
98      723554886651142144       NY 2020-02-21 02:49:31 

In [53]:
len(Valid_df)

1644

In [27]:
df = pd.DataFrame({'User_ID': user_id, 'Location': location, 'Created_time': create_time, 'Text':text})
df

,User_ID,Location,Created_time,Text
0,491923568,Louisville,2020-02-20 21:31:37,@502Heathen I was planning to be there but thi...
1,423665681,None,2020-02-20 21:31:45,Just your seasonal flu over here in the UK wit...
2,133528756,"Memphis, TN",2020-02-20 21:31:46,I’m suffering from the flu and moms won’t pick...
3,951698623951134720,Female | 🇦🇺 | 🏳️‍🌈,2020-02-20 21:31:50,Is Taemin shitting himself cause he already go...
4,964427235527544834,malaysia・eng/malay OK!,2020-02-20 21:31:52,just got my shot! i am now flu resistant
5,1227459020354703360,None,2020-02-20 21:31:54,"RT @Atheist_Krishna: एक गिलास लो, उसमे तीन आउं..."
6,1209174069112983552,None,2020-02-20 21:31:54,My work week is done .*I got extra work. This ...
7,24332297,outside Newsome's office door,2020-02-20 21:31:57,@Paige2psu I first read Pens as penis.. Okay. ...
8,2788102730,"Tampa, FL",2020-02-20 21:31:58,Braydon Coburn is on the ice for warmups. He m...
9,142876074,brasil,2020-02-20 21:32:02,RT @_soncruz: Comparar o 1 mês e meio de traba...


## Location Clean

In [28]:
US_list=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
      'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN',
      'TX','UT','VT','VA','WA','WV','WI','WY']

In [34]:
df.loc[df['Location'].str.contains('AL|Alabama|Yellowhammer ', na=False), 'Location'] = 'AL'
df.loc[df['Location'].str.contains('AK|Alaska|Last Frontier', na=False), 'Location'] = 'AK'
df.loc[df['Location'].str.contains('AZ|Arizona|Grand Canyon', na=False), 'Location'] = 'AZ'
df.loc[df['Location'].str.contains('AR|Arkansas|Natural', na=False), 'Location'] = 'AR'
df.loc[df['Location'].str.contains('CA|California|Golden', na=False), 'Location'] = 'CA'
df.loc[df['Location'].str.contains('CO|Colorado|Centennial', na=False), 'Location'] = 'CO'
df.loc[df['Location'].str.contains('CT|Connecticut|Nutmeg|Constitution', na=False), 'Location'] = 'CT'
df.loc[df['Location'].str.contains('DE|Delaware|First|Diamond', na=False), 'Location'] = 'DE'
df.loc[df['Location'].str.contains('FL|Florida|Sunshine', na=False), 'Location'] = 'FL'
df.loc[df['Location'].str.contains('GA|Georgia|Peach', na=False), 'Location'] = 'GA'
df.loc[df['Location'].str.contains('HI|Hawaii|Aloha', na=False), 'Location'] = 'HI'
df.loc[df['Location'].str.contains('ID|Idaho|Gem', na=False), 'Location'] = 'ID'
df.loc[df['Location'].str.contains('IL|Illinois|Prairie', na=False), 'Location'] = 'IL'
df.loc[df['Location'].str.contains('IN|Indiana|Hoosier', na=False), 'Location'] = 'IN'
df.loc[df['Location'].str.contains('IA|Iowa|Hawkeye|Corn', na=False), 'Location'] = 'IA'
df.loc[df['Location'].str.contains('KS|Kansas|Sunflower', na=False), 'Location'] = 'KS'
df.loc[df['Location'].str.contains('KY|Kentucky|Bluegrass', na=False), 'Location'] = 'KY'
df.loc[df['Location'].str.contains('LA|Louisiana|Pelican', na=False), 'Location'] = 'LA'
df.loc[df['Location'].str.contains('ME|Maine|Vacationland', na=False), 'Location'] = 'ME'
df.loc[df['Location'].str.contains('MD|Maryland|Old Line', na=False), 'Location'] = 'MD'
df.loc[df['Location'].str.contains('MA|Massachusetts|Bay', na=False), 'Location'] = 'MA'
df.loc[df['Location'].str.contains('MI|Michigan|Great Lake|Wolverine', na=False), 'Location'] = 'MI'
df.loc[df['Location'].str.contains('MN|Minnesota|North Star', na=False), 'Location'] = 'MN'
df.loc[df['Location'].str.contains('MS|Mississippi|Magnolia', na=False), 'Location'] = 'MS'
df.loc[df['Location'].str.contains('MO|Missouri|Show Me', na=False), 'Location'] = 'MO'
df.loc[df['Location'].str.contains('MT|Montana|Treasure', na=False), 'Location'] = 'MT'
df.loc[df['Location'].str.contains('NE|Nebraska|Cornhusker', na=False), 'Location'] = 'NE'
df.loc[df['Location'].str.contains('NV|Nevada|Silver', na=False), 'Location'] = 'NV'
df.loc[df['Location'].str.contains('NH|New Hampshire|Granite', na=False), 'Location'] = 'NH'
df.loc[df['Location'].str.contains('NJ|New Jersey|Garden', na=False), 'Location'] = 'NJ'
df.loc[df['Location'].str.contains('NM|New Mexico|Sunshine|Enchantment', na=False), 'Location'] = 'NM'
df.loc[df['Location'].str.contains('NY|New York|Empire', na=False), 'Location'] = 'NY'
df.loc[df['Location'].str.contains('NC|North Carolina|Tar Heel', na=False), 'Location'] = 'NC'
df.loc[df['Location'].str.contains('ND|North Dakota|Flickertail', na=False), 'Location'] = 'ND'
df.loc[df['Location'].str.contains('OH|Ohio|Buckeye', na=False), 'Location'] = 'OH'
df.loc[df['Location'].str.contains('OK|Oklahoma|Sooner', na=False), 'Location'] = 'OK'
df.loc[df['Location'].str.contains('OR|Oregon|Beaver', na=False), 'Location'] = 'OR'
df.loc[df['Location'].str.contains('PA|Pennsylvania|Keystone', na=False), 'Location'] = 'PA'
df.loc[df['Location'].str.contains('RI|Rhode Island|Ocean', na=False), 'Location'] = 'RI'
df.loc[df['Location'].str.contains('SC|South Carolina|Palmetto', na=False), 'Location'] = 'SC'
df.loc[df['Location'].str.contains('SD|South Dakota|Mount Rushmore', na=False), 'Location'] = 'SD'
df.loc[df['Location'].str.contains('TN|Tennessee|Volunteer', na=False), 'Location'] = 'TN'
df.loc[df['Location'].str.contains('TX|Texas|Lone Star', na=False), 'Location'] = 'TX'
df.loc[df['Location'].str.contains('UT|Utah|Beehive', na=False), 'Location'] = 'UT'
df.loc[df['Location'].str.contains('VT|Vermont|Green Mountain', na=False), 'Location'] = 'VT'
df.loc[df['Location'].str.contains('VA|Virginia|Old Dominion', na=False), 'Location'] = 'VA'
df.loc[df['Location'].str.contains('WA|Washington|DC|Evergreen', na=False), 'Location'] = 'WA'
df.loc[df['Location'].str.contains('WV|West Virginia|Mountain ', na=False), 'Location'] = 'WV'
df.loc[df['Location'].str.contains('WI|Wisconsin|Badger', na=False), 'Location'] = 'WI'
df.loc[df['Location'].str.contains('WY|Wyoming|Equality', na=False), 'Location'] = 'WY'

Valid_df = df[df['Location'].isin(US_list)]

,User_ID,Location,Created_time,Text
0,171534110,"São Paulo, Brasil",2020-02-20 21:46:18,"A brisa da corrida é o seguinte, vento no rost..."
1,1112880973782421510,Sweden,2020-02-20 21:46:18,"RT @WatchingHawks: In San Quentin Prison, Gett..."
2,190648628,InACastleDarkOrAFortressStrong,2020-02-20 21:46:24,RT @PDChina: The #COVID19 #pneumonia may turn ...
3,28053812,None,2020-02-20 21:46:26,#SARSCoV2 #COVID-19 #coronavirus
4,1018888068193894400,"Bandung, Jawa Barat",2020-02-20 21:46:28,RT @Anyaselalubenar: Penyakit di musim hujan:\...
5,1219139647374127105,Indonesia,2020-02-20 21:46:28,RT @Anyaselalubenar: Penyakit di musim hujan:\...
6,4155786082,Reino da Arábia Saudita,2020-02-20 21:46:32,@Ighffc As chance que o Flu perdeu aqui no mar...
7,153139876,Penny Lane,2020-02-20 21:46:33,"Como toca explicarles los tuits, la comparació..."
8,2668924782,"Denpasar, Bali",2020-02-20 21:46:35,"Lagi sakit, demam, flu, dibalurin minyak kayu ..."


In [46]:
Valid_df = df[df['Location'].isin(US_list)]

Valid_df

,User_ID,Location,Created_time,Text
1,1207022268578070528,CO,2020-02-20 23:34:05,@Jonewto @HiCecilia @Aiden47127829 @wolfbabes ...
6,137039322,IL,2020-02-20 23:34:17,This flu shit 🙄
14,1217542046266937345,MI,2020-02-20 23:34:27,RT @livecrisisnews: Paper someone recieved aft...
16,3307502381,AR,2020-02-20 23:34:29,I can't even sleep I keep waking up out my sle...
20,133920645,OR,2020-02-20 23:34:33,"RT @TomthunkitsMind: ""No occasion for panic""\n..."
21,3274765578,TX,2020-02-20 23:34:34,RT @RosaBrighid: My second workout since recov...
23,75207629,MD,2020-02-20 23:34:37,@pac_watson Have you personally seen any cases...
25,828445830239625218,NV,2020-02-20 23:34:42,RT @RealSaavedra: Average # of deaths per day ...
34,1870257638,WA,2020-02-20 23:34:59,RT @TheRickyDavila: Hillary Clinton warned eve...
39,3333804980,AZ,2020-02-20 23:35:10,RT @minzi8mango: @sfchronicle it's just the fl...


In [48]:
    export_csv = Valid_df.to_csv ('data_0220.csv',mode='a', index = None, header=False,encoding='utf_8_sig') 

## Json format attribute

In [213]:
d = json.loads(raw_data[0])

In [214]:
d

{'created_at': 'Thu Feb 20 23:29:35 +0000 2020',
 'id': 1230635638690635781,
 'id_str': '1230635638690635781',
 'text': 'I got the flu 😒 https://t.co/LBDMSQL6nh',
 'display_text_range': [0, 15],
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1169247040414638080,
  'id_str': '1169247040414638080',
  'name': 'Dev- Err',
  'screen_name': 'Fam0usd3v',
  'location': 'Killadelphia',
  'url': None,
  'description': '19.',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 128,
  'friends_count': 117,
  'listed_count': 0,
  'favourites_count': 2054,
  'statuses_count': 1573,
  'created_at': 'Wed Sep 04 13:53:35 +0000 2019',
  'utc_offset': None,
  'time_zone': None,
  'geo_enabled': True,
  'lang': None,

In [195]:
d['text']

'RT @__highseaxo: padre cinese, mare canadese, vive a londra, ristorante con influenza africana e parla italiano\nJEREMY TI AMO INGRAVIDAMI #…'

In [189]:
d["user"]["id"]

613425122

In [142]:
d["user"]["location"]

'hell'

In [171]:
tm=d["created_at"]
tm

'Thu Feb 20 22:37:41 +0000 2020'

In [172]:
type(tm)

str

In [173]:
import time

ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tm,'%a %b %d %H:%M:%S +0000 %Y'))

In [175]:
type(ts)

str

In [182]:
tt=datetime.strptime(ts,'%Y-%m-%d %H:%M:%S')

In [185]:
from datetime import datetime, timedelta

tweet_time = tt- timedelta(hours=5)

tweet_time.strftime('%Y-%m-%d %H:%M:%S')

'2020-02-20 17:37:41'

In [199]:
js=json.loads(data)
            user_id=js["user"]["id"]
            user_location=js["user"]["location"]
            tm=js["created_at"]
            ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tm,'%a %b %d %H:%M:%S +0000 %Y'))
            tt=datetime.strptime(ts,'%Y-%m-%d %H:%M:%S')
            tweet_time = tt- timedelta(hours=5)
            tweet_time.strftime('%Y-%m-%d %H:%M:%S')
            tweet_text=js['text']
            tweets.append({'User_ID': user_id, 'Location': user_location, 'Created_time': tweet_time, 'Text':tweet_text}, ignore_index=True)

,User_ID,Location,Create_time,Text
